In [1]:
pip install -U langchain langchain-community langchain-openai chromadb pypdf python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Disable telemetry to avoid noisy errors
os.environ["ANONYMIZED_TELEMETRY"] = "False"
os.environ["CHROMA_TELEMETRY"] = "False"

# Load .env (put it in project root or notebooks folder)
load_dotenv()

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
assert OPENROUTER_API_KEY, "Missing OPENROUTER_API_KEY in .env"

print("✅ OPENROUTER_API_KEY loaded")


✅ OPENROUTER_API_KEY loaded


In [3]:
from pypdf import PdfReader

# ✅ Put your resume path here
# Option A: if resume PDF in project root
pdf_path = Path.cwd().parent / "Tala Dweikat CV.pdf"

# Option B: if it's elsewhere, use full path:
# pdf_path = Path(r"C:\Users\user\...\Tala Dweikat CV.pdf")

assert pdf_path.exists(), f"PDF not found: {pdf_path}"

reader = PdfReader(str(pdf_path))
resume_text = "\n".join(page.extract_text() or "" for page in reader.pages)

print("✅ resume_text length:", len(resume_text))
print(resume_text[:300])


✅ resume_text length: 5160
 
tala.nazeeh.dowiekat@gmail.com                                               
Tala  Dweikat     
                         https://github.com/taladowiekat 
 www.linkedin.com/in/tala-dweikat-a80712276                                                                                                    


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=120
)

chunks = splitter.split_text(resume_text)
print("✅ chunks:", len(chunks))
print("Sample chunk:\n", chunks[0][:300])


c:\Users\user\anaconda3\envs\mini-rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ chunks: 10
Sample chunk:
 tala.nazeeh.dowiekat@gmail.com                                               
Tala  Dweikat     
                         https://github.com/taladowiekat 
 www.linkedin.com/in/tala-dweikat-a80712276                                                                                                      


In [5]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

PERSIST_DIR = str(Path.cwd().parent / "resume_db_chain")  # NEW folder
COLLECTION_NAME = "resumes_chain"

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1",
)

# Build vectorstore from scratch
vectorstore = Chroma.from_texts(
    texts=chunks,
    embedding=embeddings,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIR
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

print("✅ Vector DB created at:", PERSIST_DIR)


✅ Vector DB created at: c:\Users\user\OneDrive\Documents\Desktop\mini-rag-project\mini-rag\RAG-Retrieval-basedon-chuncking\resume_db_chain


In [7]:
docs = retriever.invoke("What is Tala's email address?")
print("docs:", len(docs))
print(docs[0].page_content[:250] if docs else "NO DOCS")


docs: 5
tala.nazeeh.dowiekat@gmail.com                                               
Tala  Dweikat     
                         https://github.com/taladowiekat 
 www.linkedin.com/in/tala-dweikat-a80712276                                                    


In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n---\n\n".join(d.page_content for d in docs)

prompt = ChatPromptTemplate.from_template("""
You are a helpful career advisor. Use ONLY the provided context to answer.

Rules:
- If the answer is not in the context, say: "I don't know based on the provided resume."
- Keep the answer concise and clear.

Context:
{context}

Question:
{question}

Answer:
""")

# LLM1
llm1 = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1",
    temperature=0
)

# LLM2 (comparison)
llm2 = ChatOpenAI(
    model="meta-llama/llama-3.1-8b-instruct",
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1",
    temperature=0
)

rag_chain_llm1 = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm1
    | StrOutputParser()
)

rag_chain_llm2 = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm2
    | StrOutputParser()
)

print("✅ RAG chains ready (LLM1 + LLM2)")


✅ RAG chains ready (LLM1 + LLM2)


In [9]:
questions = [
    "What is Tala's email address?",
    "What ML models did Tala use in the Tesla Stock Prediction project?",
    "List Tala's main technical skills in 5 bullet points.",
]

for q in questions:
    print("\n" + "="*90)
    print("Q:", q)

    print("\n--- LLM1 (gpt-4o-mini) ---")
    print(rag_chain_llm1.invoke(q))

    print("\n--- LLM2 (llama-3.1-8b) ---")
    print(rag_chain_llm2.invoke(q))



Q: What is Tala's email address?

--- LLM1 (gpt-4o-mini) ---
tala.nazeeh.dowiekat@gmail.com

--- LLM2 (llama-3.1-8b) ---
tala.nazeeh.dowiekat@gmail.com

Q: What ML models did Tala use in the Tesla Stock Prediction project?

--- LLM1 (gpt-4o-mini) ---
Tala used Random Forest, Gradient Boosting, and Linear Regression models in the Tesla Stock Prediction project.

--- LLM2 (llama-3.1-8b) ---
Random Forest, Gradient Boosting, and Linear Regression.

Q: List Tala's main technical skills in 5 bullet points.

--- LLM1 (gpt-4o-mini) ---
- C++, Python, JavaScript, HTML, CSS
- ReactJS, Node.js, Express, Bootstrap, MUI
- NumPy, pandas, Matplotlib, Pytorch, scikit-learn
- SQL, NOSQL, MongoDB, Sequelize
- Git, GitHub

--- LLM2 (llama-3.1-8b) ---
Here are Tala's main technical skills in 5 bullet points:

• Programming languages: C++, Python, JavaScript
• Frameworks & Libraries: ReactJS, Node.js, Express, Bootstrap, MUI
• Databases: SQL, NOSQL, MongoDB
• Tools: Git, GitHub
• Machine Learning: Pytorc

In [10]:
eval_prompt = ChatPromptTemplate.from_template("""
You are an evaluator for a RAG system.

Given:
- Question
- Retrieved Context
- Model Answer

Rate from 1 to 5:
1) groundedness: supported by context?
2) relevance: answers the question?
3) correctness: claims correct given context?
4) completeness: includes key points?

Return JSON ONLY:
{{
  "groundedness": <1-5>,
  "relevance": <1-5>,
  "correctness": <1-5>,
  "completeness": <1-5>,
  "notes": "short justification"
}}

Question: {question}

Context:
{context}

Answer:
{answer}
""")

judge = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1",
    temperature=0
)

def evaluate_answer(question: str, answer: str):
    ctx_docs = retriever.invoke(question)
    ctx = format_docs(ctx_docs)
    out = (eval_prompt | judge | StrOutputParser()).invoke(
        {"question": question, "context": ctx, "answer": answer}
    )
    return out


In [11]:
eval_questions = [
    "What is Tala's email address?",
    "What ML models did Tala use in the Tesla Stock Prediction project?",
]

for q in eval_questions:
    print("\n" + "="*90)
    print("EVAL QUESTION:", q)

    a1 = rag_chain_llm1.invoke(q)
    a2 = rag_chain_llm2.invoke(q)

    print("\n--- LLM1 Answer ---")
    print(a1)
    print("\n--- LLM1 Eval ---")
    print(evaluate_answer(q, a1))

    print("\n--- LLM2 Answer ---")
    print(a2)
    print("\n--- LLM2 Eval ---")
    print(evaluate_answer(q, a2))



EVAL QUESTION: What is Tala's email address?

--- LLM1 Answer ---
tala.nazeeh.dowiekat@gmail.com

--- LLM1 Eval ---
{
  "groundedness": 5,
  "relevance": 5,
  "correctness": 5,
  "completeness": 5,
  "notes": "The answer is directly supported by the context, accurately addresses the question, and includes the complete email address."
}

--- LLM2 Answer ---
tala.nazeeh.dowiekat@gmail.com

--- LLM2 Eval ---
{
  "groundedness": 5,
  "relevance": 5,
  "correctness": 5,
  "completeness": 5,
  "notes": "The answer is directly supported by the context, is relevant to the question, is correct, and includes the complete email address."
}

EVAL QUESTION: What ML models did Tala use in the Tesla Stock Prediction project?

--- LLM1 Answer ---
Tala used Random Forest, Gradient Boosting, and Linear Regression models in the Tesla Stock Prediction project.

--- LLM1 Eval ---
{
  "groundedness": 5,
  "relevance": 5,
  "correctness": 5,
  "completeness": 5,
  "notes": "The answer is fully supported by 

In [12]:
import json
import pandas as pd

rows = []
for q in eval_questions:
    a1 = rag_chain_llm1.invoke(q)
    a2 = rag_chain_llm2.invoke(q)

    e1 = json.loads(evaluate_answer(q, a1))
    e2 = json.loads(evaluate_answer(q, a2))

    rows.append({"question": q, "model": "gpt-4o-mini", "answer": a1, **e1})
    rows.append({"question": q, "model": "llama-3.1-8b", "answer": a2, **e2})

df = pd.DataFrame(rows)
df


,question,model,answer,groundedness,relevance,correctness,completeness,notes
0,What is Tala's email address?,gpt-4o-mini,tala.nazeeh.dowiekat@gmail.com,5,5,5,5,The answer directly provides Tala's email addr...
1,What is Tala's email address?,llama-3.1-8b,tala.nazeeh.dowiekat@gmail.com,5,5,5,5,The answer is directly supported by the contex...
2,What ML models did Tala use in the Tesla Stock...,gpt-4o-mini,"Tala used Random Forest, Gradient Boosting, an...",5,5,5,5,The answer directly lists the ML models used i...
3,What ML models did Tala use in the Tesla Stock...,llama-3.1-8b,Tala used the following ML models in the Tesla...,5,5,5,4,"The answer is fully supported by the context, ..."
